# Polars Tutorial
In this notebook, we'll walk through some of the most useful features of Polars.

To do so, we'll use a [dataset by Terna](https://www.terna.it/en/electric-system/transparency-report/actual-generation), describing the total generation in Italy in the last year.

## Setup

In [ ]:
from pathlib import Path

import polars as pl
import plotly.express as px
import plotly.io as pio

pio.templates.default = "plotly_white"

In [ ]:
data_path = Path("..") / "data" / "power_generation.xlsx"
df = pl.read_excel(data_path)
df.head()

# Anatomy of a Polars DataFrame
A Polars DataFrame is composed of several key elements.

Columns form the backbone of a DataFrame. Each column represents a single variable and contains data of a specific type. Polars uses Arrow arrays for efficient memory usage and fast operations.

Rows in Polars are conceptual rather than physical objects. They represent individual records but are accessed through column operations.

Data types (dtypes) in Polars include common types like integers, floats, strings, and booleans, as well as specialized types for dates, times, and categorical data. The dtype determines how data is stored and manipulated.

Unlike some other libraries, Polars doesn't use a separate index. Instead, the position of data in columns serves as an implicit index.

The schema defines the structure of the DataFrame, specifying column names and their data types. It's crucial for understanding the layout of your data.

Polars uses a chunked data structure, allowing for efficient parallel processing and lazy evaluation. This is key to its performance with large datasets.

Expressions are a powerful feature in Polars for data manipulation. They allow you to define complex operations on columns efficiently.

To work with Polars effectively, it's important to understand how these components interact. For example, you might select columns, apply expressions to transform data, or use the schema to understand your dataset's structure. The dtype of a column influences what operations you can perform, while the chunked structure enables Polars to process large datasets efficiently.

In [ ]:
print(df.describe())

In [ ]:
df.dtypes

In [ ]:
df.columns

In [ ]:
df.schema

# Basic Manipulation
We'll now demonstrate some data manipulation using Polars. For the full reference, please refer to the [official documentation](https://docs.pola.rs/)

In [ ]:
df.select(["Date", "Actual Generation (GWh)", "Primary Source"])

In [ ]:
df.rename({"Actual Generation (GWh)": "Generation", "Primary Source": "Source"})

In [ ]:
df.with_columns((pl.col("Actual Generation (GWh)") * 1000).alias("Generation_MWh"))

# Filtering and Sorting

In [ ]:
df.filter(pl.col("Primary Source") == "Thermal")

In [ ]:
df.filter(pl.col("Actual Generation (GWh)") > 10)

In [ ]:
df.sort("Actual Generation (GWh)", descending=True)

# Grouping and Aggregations


In [ ]:
df.group_by("Primary Source").agg(
    pl.col("Actual Generation (GWh)").sum().alias("Total_Generation"),
    pl.col("Actual Generation (GWh)").mean().alias("Avg_Generation"),
    pl.col("Actual Generation (GWh)").count().alias("Count"),
)

In [ ]:
df.group_by("Primary Source").agg(
    pl.col("Actual Generation (GWh)").max().alias("Max Gen")
)

# Time series operations
Many datasets in energy are time series, and our dataset is no exception. 

Therefore, we'll show some operations that can be performed on dates and time series.

In [ ]:
df.with_columns(pl.col("Date").dt.hour().alias("Hour"))

In [ ]:
generation_df = (
    df.with_columns(pl.col("Date").dt.hour().alias("Hour"))
    .group_by("Hour")
    .agg(pl.col("Actual Generation (GWh)").mean().alias("Hourly Generation"))
    .sort("Hour")
)

In [ ]:
px.line(generation_df, x="Hour", y="Hourly Generation")

In [ ]:
generation_by_hour_source_df = (
    df.with_columns(pl.col("Date").dt.hour().alias("Hour"))
    .with_columns(
        pl.col("Actual Generation (GWh)")
        .mean()
        .over("Hour", "Primary Source")
        .alias("Hourly Generation")
    )
    .select("Hour", "Primary Source", "Hourly Generation")
    .unique()
    .sort("Hour")
)

In [ ]:
px.area(
    generation_by_hour_source_df,
    x="Hour",
    y="Hourly Generation",
    color="Primary Source",
)

# Advanced Data Manipulation
We'll now showcase some operations such as pivot and rolling means.

In [ ]:
df.pivot(
    values="Actual Generation (GWh)",
    index="Date",
    on="Primary Source",
    aggregate_function="mean",
)

In [ ]:
(
    df.select(pl.col("Date"), pl.col("Actual Generation (GWh)").sum().over("Date"))
    .unique()
    .sort("Date")
    .select(pl.col("Date"), pl.col("Actual Generation (GWh)").rolling_mean(24))
)

# Exporting Data

In [ ]:
output_df = (
    df.with_columns(date=pl.col("Date").dt.date())
    .group_by("date", "Primary Source")
    .agg(pl.col("Actual Generation (GWh)").sum().alias("generation_gwh"))
    .select(
        pl.col("date"),
        pl.col("generation_gwh"),
        pl.col("Primary Source").alias("source"),
    )
    .sort("date", "source")
)

In [ ]:
output_df.write_csv("output.csv")
output_df.write_excel("output.xlsx")
output_df.write_parquet("output.parquet")